# EECS 595 HW3: Parts 1-3 Building and Pre-Training BERT from Scratch

In [68]:
import os
import math
import numpy as np
import random
import logging
import torch.nn.functional as F
# Bring in PyTorch
import torch
import torch.nn as nn

# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from copy import deepcopy
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2
#import wandb

import matplotlib.pyplot as plt

# For progress and timing
from tqdm.auto import tqdm, trange
import time

from copy import deepcopy

# Part 1: Tokenization

To start with, we'll load in one of the tokenizers for the original BERT from the `tokenizers` package. 

In [4]:
################################################################
#                     TODO: YOUR CODE HERE                     #
#
#  1. Create a new BertWordPieceTokenizer using the specified vocab.txt file in the homework. 
#     Make sure that all tokens are lowed cased.
#
################################################################
from tokenizers import BertWordPieceTokenizer

# bert tokenizer
tokenizer = BertWordPieceTokenizer("vocab.txt", lowercase=True)

# Test the tokenizer
print(tokenizer.encode("Hello, I am learning to tokenize").tokens)
print(tokenizer.encode("Hello, I am learning to tokenize. this is difficult").tokens)
print(tokenizer.encode("I like to read books").tokens)
print(tokenizer.encode("drink some whisky and have fun").tokens)
print(tokenizer.encode("I hated you").tokens)
print(tokenizer.get_vocab().keys())

['[CLS]', 'hello', ',', 'i', 'am', 'learning', 'to', 'token', '##ize', '[SEP]']
['[CLS]', 'hello', ',', 'i', 'am', 'learning', 'to', 'token', '##ize', '.', 'this', 'is', 'difficult', '[SEP]']
['[CLS]', 'i', 'like', 'to', 'read', 'books', '[SEP]']
['[CLS]', 'drink', 'some', 'whisky', 'and', 'have', 'fun', '[SEP]']
['[CLS]', 'i', 'hated', 'you', '[SEP]']


AttributeError: 'dict_keys' object has no attribute 'size'

In [70]:
# check if gpu is available
device = 'cpu' 
if torch.backends.mps.is_available():
    device = 'mps'
if torch.cuda.is_available():
    device = 'cuda'
print(f"Using '{device}' device")

Using 'mps' device


# Part 2: Building a Transformer Encoder and BERT

The BERT model is a pre-trained transformer network. Creating a small BERT model will take two big piecse (1) building the transformer itself and (2) writing the code that incorporates the tranformer into BERT and has it set up for training. We'll try to simplify building this by thinking of the different pieces as building blocks that we can put together. Remember, all neural networks are _functions_ and you can compose functions together to make a new function. 

Let's take a look at the overall diagram for Transformers/BERT:
![The diagram of the transformer newtork](https://devopedia.org/images/article/235/5113.1573652896.png)
The trickiest part is the left where we need to deal with the scaled dot-product attention. You've already seen attention though in Homework 2, so some of this should be familiar. 

You'll implement the following pieces to put it all together.

Steps:
1. Embedding: BERT will learn word embeddings that are similar to word2vec's _but_ also incorporate the position of the embedding in the sequence
2. Multi-headed Attention: The core part of the network that learns how much each token should pay attention to all other tokens
3. A Feed-forward Layer: The layer that transformers the attention-combined representations
4. The Transformer Encoder: The unified transformer network that combines attention with the feed-forward layer
5. A BERT Classification Layer: The classificiation part of BERT
6. A BERT Masked Language Modeling (MLM) Layer: The part of BERT that deals with MLM training
7. The overall BERT model: The final BERT model architecture that supports both MLM and Classification

Feel free to read the instructions for all steps in Part 2 before getting started to see how the pieces might fit together. To get everything working, you'll need all parts, which build on each other, so we recommend starting with the first and moving on from there.

## Part 2.1: Embedding Layer
BERT's input embeddings are normally the sum of three embeddings:
- Token Embeddings: The input token embeddings
- Position Embeddings: The position of the token in the sequence
- Token Type Embeddings: The segment (sentence) the token belongs to

The second piece helps BERT learn to distinguish that the same token is in different positions. Remember the attention mechanism works independently of where each of the tokens are; without positional information added to the word embedding, the model can't distinguish between words in different orders!

The third piece was designed for the Next Sentence Prediciton (NSP) task during pre-training. Here, two sentences are provided as input with the special `[SEP]` token between them. The NSP task is a classifiction task based on whether the two sentences did or did not actually follow each other. Just like in word2vec, we would sample random sentences as not-next. The hope for this pretraining task was that it would help BERT learn discourse coherence. However, some later works have shown NSP doesn't actually help that much and training time is probably better spent on doing more MLM, so some more advanced models dropped this.

For simplicity, in Homework 3, you only need to deal with token embeddings and positional embeddings, but do not need to deal with token-type embeddings.

**NOTE:** When talking about embeddings, people (and these instructions) will talk about various things being embedded, e.g., words, tokens, wordpieces, subwords, etc. In practice, these are all based on whatever the tokenizer is producing, and BERT (and you) is agnostic to what is actually being embedded. When the instructions talk about "token embeddings" this are still just the output of the BERT WordPiece tokenizer.

In [71]:
class BertPositionalEmbedding(nn.Module):
    def __init__(self, vocab_dim: int, 
                    hidden_dim: int = 768, 
                    padding_idx: int = 0, 
                    max_seq_length: int = 512):
        
        super().__init__()

        '''
        Initialize the Embedding Layers
        '''
        self.hidden_dim = hidden_dim
        self.padding_idx = padding_idx
        self.max_seq_length = max_seq_length
    

        ################################################################################
        #                             TODO: YOUR CODE HERE                             #
        #                                                                              #
        #  1. Create two Embedding objects for the words and the positions.            #
        #     For the word embeddings keep track of which index is the padding index.  #        
        #                                                                              #
        ################################################################################
        

        self.word_embeddings = nn.Embedding(vocab_dim, self.hidden_dim, padding_idx=self.padding_idx)
        self.positional_embeddings = nn.Embedding(self.max_seq_length, hidden_dim)

    def forward(self, token_ids: torch.Tensor
                ) -> torch.Tensor:
        
        
       

        '''
        Define the forward pass of the Embedding Layers
        '''
        
        ############################################################################
        #                               TODO: YOUR CODE HERE                       #
        #                                                                          #
        # 1. Look up the relevant token embeddings from the word_embeddings layer  #
        # 2. Return the sum of the token embeddings and the positional embeddings  #
        #                                                                          #
        ############################################################################
        len_sentence = token_ids.shape[-1]# reamain modifying
        # token_ids = [bsz, len_sen]
        

        position_ids = torch.tensor(list(range(len_sentence))).to(device)

        #print(position_ids)
        #print(self.word_embeddings(token_ids).shape)
        word = self.word_embeddings(token_ids)
        position = self.positional_embeddings(position_ids)
        return position + word



## Part 2.2: Multi-Head Attention

This is the trickiest part of the homework where we implement the attention part of the transformer. Let's take a look at the attention:

![The attention network](https://www.tutorialexample.com/wp-content/uploads/2021/03/The-structure-of-Multi-Head-Attention.png)




In [72]:
# Optional TODO: Try implementing attention with some embeddings to get a sense of the core parts
# Note that this is not a complete implementation of the attention mechanism in the transformer,
# which also includes some scaling and masking operations as well as dealing with multiple heads.

# Generate some Q, K, V embeddings for a batch of 10 sequences of length 4 with embedding size 7
q_emb = torch.randn(10, 4, 7)
k_emb = torch.randn(10, 4, 7)
v_emb = torch.randn(10, 4, 7)

# Start by computing the dot product of the Q and K embeddings
dot_prod = (q_emb @ k_emb.transpose(-1, -2)) 

# Then apply a softmax to get the attention weights
attn = nn.functional.softmax(dot_prod, dim=-1)

# This should be torch.Size([10, 4, 4]) --- i.e., how much each word (4 words) pays attention to each other word
print(attn.shape)

# Now compute the weighted words by multiplying the attention weights by the V embeddings
weighted_output = attn @ v_emb

# This should be torch.Size([10, 4, 7]) --- i.e., the same shape as our inputs, but the embedings are weighted combinations of the V embeddings!
print(weighted_output.shape)

torch.Size([10, 4, 4])
torch.Size([10, 4, 7])


In [73]:
# Optional TODO: How to implicitly represent attention weights in a single layer

# Let's start with a single projection of the input embedding for Q for a sequence of 3 words with embedding size 8
# NOTE: we'll use randint here to make it easier to see the reshaping when we print the tensors
q_emb = torch.randint(10, (3, 8))
print(q_emb)
# If we have two attention heads we can reshape this tensor so that the first dimension is the number of heads
q_s = q_emb.shape
# Should be torch.Size([3, 8])
print(q_s)
n_heads = 2
multihead_q_emb = q_emb.view(n_heads, q_s[0], q_s[1]//n_heads)

# Should be torch.Size([2, 3, 4])
print(multihead_q_emb.shape)

# Check that the reshaping is correct by looking at which values when where
print(multihead_q_emb)

# Remember: Each head has its own attention! So we'll need to use this kind of reshaping for K and V so we can evantually compute
# the per-head specifici attention weights. 

# Optional TODO: try creating the K projection and re-shaping it and then the calculate its attention weights using the logic from the above cell
# Note that you'll now have another dimension in the tensor! 

tensor([[5, 5, 9, 8, 9, 9, 6, 8],
        [2, 3, 8, 9, 4, 2, 3, 2],
        [2, 6, 9, 0, 9, 2, 4, 1]])
torch.Size([3, 8])
torch.Size([2, 3, 4])
tensor([[[5, 5, 9, 8],
         [9, 9, 6, 8],
         [2, 3, 8, 9]],

        [[4, 2, 3, 2],
         [2, 6, 9, 0],
         [9, 2, 4, 1]]])


In [74]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, hidden_size: int, num_heads: int):
        '''
        Arguments:
        hidden_size: The total size of the hidden layer (across all heads)
        num_heads: The number of attention heads to use
        '''

        super().__init__()

        '''
        Initialize the Multi-Headed Attention Layer
        '''

        ###########################################################################################################################
        #                                                     TODO: YOUR CODE HERE                                                #
        #                                                                                                                         #
        # 1. Figure out how many dimensions each head should have                                                                 #
        # 2. Create linear layers to turn the input embeddings into the query, key, and value projections                         #
        # 3. Calculate the scale factor (1 / sqrt(per-head embedding size))                                                       #
        #                                                                                                                         #                                            
        # NOTE: Each of the Q, K, V projections represents the projections of *each* of the heads as one long sequence.           #
        #       Each of the layers is implicitly representing each head in different parts of its dimensions. E.g., if you        #
        #       have 4 heads and 16 dimensions, the first 4 dimensions are the first head, the second 4 dimensions are            #
        #       the second head, etc.                                                                                             #                        
        ###########################################################################################################################
    
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_embedding_size = hidden_size // num_heads
        self.scale_factor = 1 / np.sqrt(self.head_embedding_size)

        # Q, K, V
        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)

    def forward(self, embeds: torch.Tensor,  # = [bsz, T_len, embedding_size]
                mask: Optional[torch.Tensor] = None
                ) -> torch.Tensor:
        '''
        Arguments:
        embeds: The input embeddings to compute the attention over
        mask: A boolean mask of which tokens are valid to use for computing attention (see collate below)
        '''

        #####################################################################################################################################################
        #                                                   TODO: YOUR CODE HERE                                                                            #
        #                                                                                                                                                   #
        # This is the hard part of the assignment where you'll need to implement the multi-headed attention mechanism.                                      #
        # The cell above has the core logic for the attention mechanism to get you started with getting the tensor                                          #
        # shapes lined up correctly. We recommend working through that manually with some small examples to get a sense                                     #
        # of what is happening at each step.                                                                                                                #
        #                                                                                                                                                   #
        # One of the key parts to work through is how to handle the shapes of the different embeddings. For attention to work                               #
        # and be efficient, we'll only need to do a few matrix multiplications and a softmax. The key is to figure out how to                               #
        # do this by getting the tensors into the right shapes. We strongly recommend trying to write comments at each step                                 #
        # That describe the shape of the tensors at each step in terms of what each representings.   This will help you understand                          #
        # what is happening and debug.                                                                                                                      #
        #                                                                                                                                                   #
        # We recommend using notation like the following which you'll also see in papers and blogs:                                                         #
        #  - B: Batch size                                                                                                                                  #
        #  - H: Number of heads                                                                                                                             #
        #  - T: Sequence length                                                                                                                             #
        #  - D: Embedding size                                                                                                                              #
        #                                                                                                                                                   #
        # 1. Figure out what are the dimensions of the input embeddings and which dimensions                                                                #
        #    represent what (e.g., the batch size, sequence length, etc.)                                                                                   #
        # 2. Project the input embeddings into the Q, K, and V spaces                                                                                       #
        # 3. Compute the attention weights from the Q and K projections (be sure to scale the dot product by the scale factor!)                             #
        # 4. *If their is a mask*, apply the mask to the attention weights where masked values are set to -inf                                              #
        # 5. Compute the weighted sum of the V embeddings using the attention weights                                                                       #
        # 6. Return the re-weighted output values *and* the attention weights in the shape (Batch, Heads, SeqLen, SeqLen)                                   #
        #    We'll use the attention weights for visualization later                                                                                        #
        #                                                                                                                                                   #
        # NOTE: when we say dimension, we're referring to how many axes are in a tensor. E.g., a vector                                                     #
        #       is a one-dimensional tensor, a matrix is a two-dimensional tensor, etc. Each dimension has a size too (number of components),               #
        #       e.g., a 3x4 tensor has 2 dimensions, where the first has 3 elements/components and the second has 4 elements/components.                    #
        # NOTE: You will probably want to use the .view() method to reshape the input embeddings with respect to the number of heads                        #
        #       so that you can get a tensor where one dimension corresponds to each head                                                                   #
        # NOTE: You may want to use the .transpose() method to swap dimensions (e.g., to move the heads dimension to the front)                             #
        # NOTE: Check out masked_fill for applying the mask to the attention weights                                                                        #
        # NOTE: You will not need to concatenate anything in practice because the Q, K, V projections already represent concatenated head-specific values   #
        #    in the right dimensions.                                                                                                                       #
        # HINT: You can reshape any tensor with view to "add a dimension" to it.                                                                            #
        #    E.g., if you have a tensor with shape (B, T, D) and you want to add a dimension to the front, you can do tensor.view(B, 1, T, D)               #
        #####################################################################################################################################################
        B, T, D = embeds.shape
        
        Q = self.q_proj(embeds)
        K = self.k_proj(embeds)
        V = self.v_proj(embeds)#([bsz, T_len, embedding_size])

        Q = Q.view(B, self.num_heads, T, self.head_embedding_size)
        K = K.view(B, self.num_heads, T, self.head_embedding_size)
        V = V.view(B, self.num_heads, T, self.head_embedding_size)
        #[bsz, heads, T_len, embedding_per_heads]
        att_QK = Q @ K.transpose(-2,-1) * self.scale_factor

        if mask is not None:
            mask = mask.view(B,1,1,T).to(device)
            att_QK = att_QK.masked_fill(mask == 0, float('-inf'))# mask == false -> -100
        #att_QK += mask_3d
        # if mask is not None: # mask_fill
        #     mask_mat = torch.triu(torch.ones(B, self.num_heads, T, self.head_embedding_size)) * float('-inf') 
        #     att_QK += mask_mat

        att_weight = torch.softmax(att_QK, dim = -1)
        #[bsz, heads, T_len, T_len]
        att = att_weight @ V
        #[bsz, heads, T_len, embedding_per_heads]

        
        return att, att_weight







        

        
         









## Part 2.3 Define the Feed-Forward Layer

The feed forward layer is a simple two-layer feed forward network (FFN) with an activation function between the layers. This network usually follows the multi-headed attention output that allows its content representation to be tranformed and aggregated. Below, we'll make a function that return this network using `nn.Sequential` which takes in a tuple or list of layers and activation functions where any input is passed through each function in order and then the output is returned.

In a tranfsormer, typically the FFN is wider than the embedding size, usally with an expansio factor of 4. This increased number of neurons allows the model to capture more interactions between dimensions of the embedding.


In [75]:
def feed_forward_layer(
    hidden_size: int, 
    feed_forward_size: Optional[int] = None, 
    activation: nn.Module = nn.GELU()
):
    '''
    Arguments:
      - hidden_size: The size of the input and output of the feed forward layer. 
      - feed_forward_size: The size of the hidden layer in the feed forward network. If None, defaults to 4 * hidden_size. This size
        specifies the size of the middle layer in the feed forward network.
      - activation: The activation function to use in the feed forward network

    Returns: 
    '''
    ################################################################
    #                     TODO: YOUR CODE HERE                     #
    # Implement the feed forward layer as described in the slides  #
    # The feed forward layer is a simple three-layer neural network#
    # with an activation function.                                 #
    #                                                              #
    # NOTE: It maps from hidden_size to feed_forward_size and then #
    #       back to hidden_size.                                   #
    ################################################################
   
    if feed_forward_size == None:
        feed_forward_size = hidden_size * 4

    feed_forward_sequence = nn.Sequential(
        nn.Linear(hidden_size, feed_forward_size),
        activation,
        nn.Linear(feed_forward_size, hidden_size)
    )
    return feed_forward_sequence

    
    





## Part 2.4 Building a Transformer Block as an Encoder Layer

Let's finish putting together our tranformer pieces into a single network that (1) computes the self-attention to get contextualized word representations and (2) passes those representations through our feed-forward neural network layers. 

During training, we'll also add some probability of using [dropout](https://machinelearningmastery.com/using-dropout-regularization-in-pytorch-models/) which is also implemented in pytorch. Dropout will randomly zero-out some values when training so the model learns to be robust to the value of any one neuron. In practice, when you switch between `train()` and `eval()`, under the hood, this will turn on/off things like dropout automatically!

In [76]:
class TransformerEncoderLayer(nn.Module):

    def __init__(
        self,
        hidden_size: int = 256, # NOTE: normally 768, but keep it small for homework
        num_heads: int = 12,
        dropout: float = 0.1,
        activation: nn.Module = nn.GELU(),
        feed_forward_size: Optional[int] = None,
    ):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.dropout = dropout
        #self.activation = activation
        self.feed_forward_size = feed_forward_size 
        

        ################################################################
        #                     TODO: YOUR CODE HERE                     #
        # Now we can put it all together to create one layer of the    #
        # transformer encoder.                                         #
        #                                                              #
        # 1. Create a MultiHeadedAttention layer with the specified    #
        #    number of heads.                                          #
        # 2. Create a feed forward layer with the specified activation #
        #    function.                                                 #
        # 3. Save the hidden_size, dropout, and feed_forward_size      #
        #    as attributes.                                            #
        # 4. Define a forward method that takes in an input tensor     #
        #    and an optional mask tensor and returns the output tensor #
        #    and the attention weights that go through the multi-      #
        #    headed attention layer and the feed forward layer.        #
        ################################################################
        
        self.MultiHeadedAttention_layer = MultiHeadedAttention(hidden_size, num_heads)

        self.feed_forward_layer = feed_forward_layer(hidden_size, feed_forward_size, activation)
        
        
        #self.dropout_layer = nn.Dropout(dropout)

        

    def maybe_dropout(self, x: torch.Tensor) -> torch.Tensor:
        
        #nn.Dropout could be set automatically when calling eval() or train()
        #F.dropout could should be set manually
        
        


        ################################################################
        #                     TODO: YOUR CODE HERE                     #
        # Implement the dropout layer to be used in the forward pass   #
        # of the transformer encoder layer (if dropout was specified)  #
        ################################################################
        
        if self.training == True:
            dropped_x = F.dropout(x, p=self.dropout)

            return dropped_x
        return x

        
        

    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None):
        '''
        Returns the output of the transformer encoder layer and the attention weights from the self-attention layer
        '''
        
        ################################################################
        #                     TODO: YOUR CODE HERE                     #
        #
        # 1. Pass the input tensor through the self-attention layer    
        # 2. Call maybe_dropout on the output of the self-attention
        # 3. Pass the output of the self-attention through the feed forward network
        # 4. return the output of the feed forward network and the attention weights
        #
        ################################################################
        #input is [bsz, T_len, embedding_size]
        B , T , D = x.shape
        att, att_weight = self.MultiHeadedAttention_layer(x, mask)#[bsz, heads, T_len, embedding_per_heads]
        _, H, _, DH = att.shape
        #print(att.shape)
        #print(att_weight)
        dropped_att = att + self.maybe_dropout(att)

        #adjust size of att
        dropped_att = dropped_att.view(B, T, D)
        
        out_ffn = dropped_att + self.maybe_dropout(self.feed_forward_layer(dropped_att))

        #out_ffn = out_ffn.view(B, H, T, DH)
        return out_ffn, att_weight






        



## Part 2.5: Masked Language Modeling Head

A BERT model usually comes with multiple "heads" that are used for different tasks. One of these heads is used for the masked language model (MLM) task during pretraining. 
        
In practice, a "head" is just a linear layer that maps the hidden size to the output size. This linear layer allows the model to adapt the token representations to a particular task, while keeping the core transformer model parameters the same across tasks. A model could have multiple heads for different tasks.  In our implementation, we'll have two heads, one for the masked language modeling task and a second for classification.

The MLM head maps the contextualized token embedding to the vocabulary, so if we have some embedding $e_i$, we're learning a weight matrix $W_{mlm}$ of size $|e_i| \times |V|$. In BERT pre-training, this weight matrix is said to be _tied_ to the input embeddings; in practice, that means we use the same weights from the `Embedding` (!), except this time the weights are used as a linear layer! (NOTE: This means you're not defining a separate linear layer). You might also see this called "parameter sharing" where the same parameters (i.e., weights) are used in different parts of the neural network. 

In [77]:
class MLMHead(nn.Module):
    def __init__(self, word_embeddings: nn.Embedding):
        '''
        Arguments:
            word_embeddings: The word embeddings to use for the prediction
        '''

        super().__init__()
        

        self.word_embeddings = word_embeddings 

    def forward(self, x):
        '''
        x: The input tensor to the MLM head containing a batch of sequences of
           contextualized word embeddings (activations from the transformer encoder 
           layers)
        '''
        
        ################################################################
        #                 TODO: YOUR CODE HERE                         #
        #                                                              #
        # The MLM head is used to predict the original token from      #
        # the masked token. The prediction is over the whole 
        # vocabulary so we'll need an activation. To make this work,
        # we'll generate a tensor the length of the vocabulary size
        # that we can push through a softmax to get the probabilities
        # of each token being present.
        #                                                              #
        # NOTE: The head should not have an activation function.       #
        # NOTE: The head should be tied to the input embeddings (i.e., #
        #       the head should map the embeddings to the vocab size). #
        #       In other words, the MLM head directly predicts the     #
        #       token from the learned embeddings instead of the       #
        #       last hidden states.                                    #
        # HINT: You can get the tensor of the word embeddings to use   #
        #       for prediction from the word_embeddings object.        #
        # HINT: Desipte all this writing, this function is only a 
        #       single line of code.                                   #
        ################################################################
        
        # x = [B,T,D]
        # word_embedding = [ vocab_size, D]
        #print('mlm begin')
        #print((x @ self.word_embeddings.weight.transpose(0,1)).shape)
        return x @ self.word_embeddings.weight.transpose(0,1)#[B, T, vocab]
        


## Part 2.6: Classification Pooler Head

When we pre-train, we'll learn a special embedding called `[CLS]` that is the first embedding in any sequences and loosely approximates the overall meaning/semantics of the input text. Frequently, when we fine-tune a BERT classifier, we're using this `[CLS]` token as the summary of the input and updating the weights accordingly. 

Here, we'll create a network that will add a `Linear` layer on top of the `[CLS]` token which can be fine-tuned to do classification later. This linear layer allows use to adapt/project the `[CLS]` to a representation more suitable for classification.  This is the classification head of BERT.

We'll use this network later when defining the `BERT` class below.

In [78]:
class Pooler(nn.Module):
    def __init__(self, hidden_size: int = 768):
        super().__init__()

        ################################################################
        #                     TODO: YOUR CODE HERE                     #
        # The BERT model usually uses the first token of the sequence  #
        # to represent the entire sequence (this is the [CLS] token).  #
        #                                                              #
        # The pooler layer is a simple linear layer that maps the      #
        # representation of the [CLS] token to the hidden size.        #
        #                                                              #
        # This pooled representation is used as the input to the       #
        # classification layer defined in the later cell.              #
        ################################################################

        self.dense_layer = nn.Linear(hidden_size, hidden_size)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        ################################################################
        #                     TODO: YOUR CODE HERE                     #
        #
        # 1. Pass the [CLS] embedding through the dense layer 
        # 2. Pass the output of the dense layer through the activation
        # 3. Return the output of the activation
        #
        # OPTIONAL TODO: One other way you can represent the contents of 
        #             the entire sequence is to use the mean of all the non-special
        #             token embeddings (also known as "mean pooling"). 
        #             You can try implementing that here and add a flag to the
        #             Pooler __init__ function to switch between the two approaches.
        #
        ################################################################
        
        input = x[:,0,:]#cls

        out = self.dense_layer(input)
        
        out = torch.tanh(out)#[B,1,hidden_size]

        return out

## Part 2.7: The BERT network!

Finally! We have arrived at putting all the pieces together into a single neural network. Our BERT model will have the main attention plus feed-forward network components and then two heads: one for MLM and one for Classification. At model creation time, we'll specify which "mode" the model should be in (MLM or classification).

This implementation is a good reminder that neural networks (e.g., `nn.Module`) are just functions on inputs. We can compose and stack them together to get some really useful (and cool) outputs as a result.

In [79]:
class BERT(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        padding_idx: int = 0,
        hidden_size: int = 768,
        num_heads: int = 12,
        num_layers: int = 12,
        dropout: float = 0.1,
        activation: nn.Module = nn.GELU(),
        feed_forward_size: Optional[int] = None,
        mode: str = "mlm",
        num_classes: Optional[int] = None,
        
    ):
        '''
        Defines BERT model architecture. Note that the arguments are the same as the default
        BERT model in HuggingFace but we'll be training a *much* smaller model for this homework.

        Arguments:
        vocab_size: The size of the vocabulary (determined by the tokenizer)
        padding_idx: The index of the padding token in the vocabulary (defined by the tokenizer)
        hidden_size: The size of the hidden layer and embeddings in the transformer encoder
        num_heads: The number of attention heads to use in the transformer encoder
        num_layers: The number of layers to use in the transformer encoder (each layer is a TransformerEncoderLayer)
        dropout: The dropout rate to use in the transformer encoder (what % of times to randomly zero out activations)
        activation: The activation function to use in the transformer encoder
        feed_forward_size: The size of the hidden layer in the feed forward network in the transformer encoder. If None, defaults to 4 * hidden_size
        mode: The mode of the BERT model. Either "mlm" for masked language modeling or "classification" for sequence classification
        num_classes: The number of classes to use in the classification layer.
        '''


        super().__init__()

        self.num_layers = num_layers
        self.mode = mode
        self.num_heads = num_heads

        


        ################################################################
        #                     TODO: YOUR CODE HERE                     #
        # Now we can put it all together to create the BERT model.     #
        #                                                              #
        # A BERT model is just a stack of transformer encoder layers   #
        #                                                              #
        # followed by a pooler layer and a classification layer.       #
        # 1. Create a BertPositionalEmbedding layer with the specified #
        #    vocab size, hidden size, and padding index.               #
        # 2. Create a stack of transformer encoder layers with the     #
        #    specified number of layers, hidden size, number of heads, #
        #    dropout, activation function, and feed forward size.      #
        # 3. Create an MLMHead layer with the specified vocab size and #
        #    padding index.                                            #
        # 4. Create a Pooler layer with the specified hidden size.     #
        # 5. Create a classification layer with the specified number   #
        #    of classes.                                               #
        #
        # HINT: you can use nn.ModuleList to stack layers.  
        #
        ################################################################
        self.Wordembedding_layer = BertPositionalEmbedding(vocab_size, hidden_size,padding_idx)
        transformer_encoder_layer = TransformerEncoderLayer(hidden_size, num_heads, dropout, activation, feed_forward_size)
        self.layers_stack = nn.ModuleList([ deepcopy(transformer_encoder_layer  )for _ in range(self.num_layers)])

        self.masked_language_model = MLMHead(self.Wordembedding_layer.word_embeddings)
        self.next_sentence_pooler = Pooler(hidden_size)

        if num_classes != None:

            self.classification_layer = nn.Linear(hidden_size, num_classes)

        

        #initialize
        for module in self.modules():
            self.init_layer_weights(module)


    def forward(
        self, 
        x: torch.Tensor, 
        mask: Optional[torch.Tensor] = None, 
    ) -> torch.Tensor:
        '''
        arguments:
        x: The input token ids
        mask: The attention mask to apply to the input (see the collate function below)
        '''
        
        ################################################################
        #                     TODO: YOUR CODE HERE                     #
        #
        # 1. Get the embeddings for the input token ids
        # 2. Calculate the attention weights for each layer (be sure to save the attention weights to return)
        # 3a. If the mode is "mlm", pass the embeddings through the MLM head and return the output
        # 3b. If the mode is "classification", pass the embeddings through the classification head
        # 4. Return the output (from the relevant head) and the attention weights
        #
        ################################################################

        


        token_embedding = self.Wordembedding_layer(x)#token_ids position_ids
        # [bsz, T_len, embedding_size]

        B, T, D = token_embedding.shape
        input_layer = token_embedding
        att_weight_col = []
        for  layer in self.layers_stack:
            
            out_ffn, att_weight = layer(input_layer, mask)#input is [bsz, T_len, embedding_size]
            #[bsz, heads, T_len, embedding_per_heads], #[bsz, heads, T_len, T_len]

            input_layer = out_ffn
            #input_layer = input_layer.view(B, T, D)#这里可以提速

            att_weight_col.append(att_weight)
            #print(input_layer.shape)
        #print('multiple transformer finished')
        if self.mode == 'mlm':
            output = self.masked_language_model(input_layer) 
            #           (B,T,vocab)
        
        if self.mode == 'classification':
            output = self.next_sentence_pooler(input_layer)#[B,1,hidden_size]
            output = self.classification_layer(output)#(B,1,num_classes)

        return output , att_weight_col









    def init_layer_weights(self, module):
        ################################################################
        #                     TODO: YOUR CODE HERE                     #
        #
        #  Initialize the weights of the model with mean 0 and std 0.02
        # 
        ################################################################
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.constant_(module.bias.data, 0)
        if isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight.data, mean=0.0, std=0.02)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()

        

In [80]:
# '''
# We can verify that the model is working by running a quick test.
# '''

sentence = "The quick brown fox jumps over the lazy dog."
tokens = tokenizer.encode(sentence)
# to tensor
token_ids = torch.tensor(tokens.ids).to(device)
bert = BERT(vocab_size=tokenizer.get_vocab_size(), 
            hidden_size=768, 
            num_heads=4, 
            num_layers=2,
        ).to(device)

attention_mask = (token_ids != 0).float()

output, attn = bert(token_ids.unsqueeze(0), attention_mask.unsqueeze(0))

print(bert)

BERT(
  (Wordembedding_layer): BertPositionalEmbedding(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (positional_embeddings): Embedding(512, 768)
  )
  (layers_stack): ModuleList(
    (0-1): 2 x TransformerEncoderLayer(
      (MultiHeadedAttention_layer): MultiHeadedAttention(
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (feed_forward_layer): Sequential(
        (0): Linear(in_features=768, out_features=3072, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=3072, out_features=768, bias=True)
      )
    )
  )
  (masked_language_model): MLMHead(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
  )
  (next_sentence_pooler): Pooler(
    (dense_layer): Linear(in_features=768, out_features=768, bias=True)
  )
)


# Part 3: Training

After all that implementation, it's time to pre-train our BERT model. We'll focus specifically on MLM training and then in a separately notebook you'll used the pre-trained BERT for Parts 5 and 6. 

To pre-train, we'll need to accomplish three pieces:
1. First, we'll need to create a `Dataset` class that says how to load and process our text data for MLM training. 
2. Second, we'll need to create a `collate` function that tells the `DataLoader` how to combine multiple training examples from our dataset into a batch. The `collate` function is critical because not all of our input texts have the same size (different sequence lengths) which will create a wrinkle for giving a model a single `Tensor`
3. Third, we'll write the core training loop.

Each of the pieces below goes over more details. As you progress as a practitioner, you'll frequently need to write `Dataset` and `collate` functions for more bespoke kinds of training tasks. This part of the assignment is intended to show you how to do some simple implementations that you can re-use later. You'll likely reuse some/all of this code in Parts 5 and 6 when setting up these functions for classification!

## Part 3.1: Create A Dataloader for Masked Language Modeling

A `Dataset` is kind of like a glorified `list` object in python. The Dataset class helps us to load and process the data, which providing functionality that lets us access the data

The main function of the Dataset class is to get the number of samples and to get a sample from the dataset. The core functionality you'll need to implement in this part of the assignment is the following:
- Load the data and tokenize it using the tokenizer.
- The __len__ method to return the number of samples in the dataset.
- The __getitem__ method to return a sample from the dataset.

As a sidenote, the `Dataset` class provides a very important abstraction for training and more sophisticated `Dataset` implementations will do usfeul things like keep only some of the dataset in memory and proactively fetch data from desk to keep the overall memory footprint low. Others may do pre-processing on the fly to avoid having large `Tensor` objects in memory (e.g., loading images and preparing them for image-based learning). Just think of how large some datasets might get---as a practitioner, these implementations are critical for efficient training!

In [81]:
class MLMDataset(Dataset):
    def __init__(self, tokenizer, data: list[str], max_seq_length=128, mlm_probability=0.15):

        ##################################################################################################################
        # TODO: YOUR CODE HERE 
        # 
        # 1. Store the arguments as fields
        #                                                                                                                #                                    #
        ##################################################################################################################

        self.tokenizer = tokenizer
        self.data = data
        self.mlm_probability = mlm_probability
        self.max_seq_length = max_seq_length
        self.tokenized_text = []

    def __len__(self):
        return len(self.data)
    
    def tokenize(self):
        '''
        Tokenizes the text in self.data, performing any preprocessing and storing the tokenized data in a new list.
        '''
        

        ##################################################################################################################
        # TODO: YOUR CODE HERE 
        #                                                                                                                #                                    #
        # 1. Tokenize the data using the tokenizer.                                                                      #
        # 2. Truncate the sequence to the maximum sequence length.                                                       #
        # 3. Add the tokenized data to a list.                                                                           #
        #
        # NOTE: To save memory, you can delete self.data after tokenizing since you'll have the copy of 
        #       the tokenized data (as ids) and won't need the raw text later.
        #
        ##################################################################################################################
        for text in self.data:

            

            text_tokenized = tokenizer.encode(text).tokens
            

            if len(text_tokenized) > self.max_seq_length:
                text_tokenized = text_tokenized[:self.max_seq_length]
            
            else :
                padding_len = self.max_seq_length - len(text_tokenized)
                # print(padding_len *['PAD'])
                # print(text_tokenized)
                text_tokenized = text_tokenized + padding_len *['[PAD]']

            self.tokenized_text.append(text_tokenized)
        
        

    def __getitem__(self, idx):
        '''
        Returns the list of the token ids of an instance in the dataset and a list of the labels for MLM (one label per token).
        '''
        labels = []
        input_ids = []
        
        
        text = self.tokenized_text[idx]
        id_mask = tokenizer.token_to_id('[MASK]')
        

        for word in (self.tokenized_text[idx]):
            
            id = tokenizer.token_to_id(word)
            
            
            
            if word == '[CLS]' or word == '[SEP]' or  word == '[MASK]':
                labels.append(-100)
                input_ids.append(id)

            elif word == '[PAD]' :
                labels.append(-100)
                input_ids.append(id)

            
                
            else:

                if random.random() < self.mlm_probability:
                    labels.append(id)
                    input_ids.append(id_mask)
                else:
                    labels.append(-100)
                    input_ids.append(id)


        
        
        return input_ids, labels




        ################################################################
        #                     TODO: YOUR CODE HERE                     #
        #
        # 1. Get the tokenized data at the specified index
        # 2. Create the mask (i.e., which words the model has to predict). Special tokens are never masked.
        #    Non-masked tokens should be set to -100, which will be ignored in the loss function. 
        #    The masked tokens should be set to the original token ids. Use the specified masking probability.        
        # 
        # Hint: Use the tokenizer's functions to get IDs as needed
        ################################################################
        

In [82]:
print(tokenizer.encode("Hello, I am learning to tokenize").tokens)
print(len(tokenizer.get_vocab()))

['[CLS]', 'hello', ',', 'i', 'am', 'learning', 'to', 'token', '##ize', '[SEP]']
30522


In [83]:
'''
Verify that the dataset is working by running a quick test.
'''

# create a fake dataset using 26 aphabets, 1000 sentences, 10-20 words per sentence randomly
data = [' '.join([chr(97 + i) for i in range(random.randint(10, 20))]) for _ in range(1000)]
#print(data)

dataset = MLMDataset(tokenizer, data)
dataset.tokenize()
#print(dataset.tokenized_text)

# get the first item
input_ids, labels = dataset[0]
print('input_ids:', input_ids)
print('labels:', labels)

input_ids: [101, 1037, 1038, 1039, 1040, 103, 103, 103, 1044, 1045, 103, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
labels: [-100, -100, -100, -100, -100, 1041, 1042, 1043, -100, -100, 1046, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -

In [84]:
## tet with one sentence
data = ["The quick brown fox jumps over the lazy dog.",
        "The quick brown fox jumps over the lazy dog.",
        "The quick brown fox jumps over the lazy dog.",]
dataset = MLMDataset(tokenizer, data)
dataset.tokenize()
print(dataset[0])


([101, 1996, 4248, 2829, 103, 14523, 103, 1996, 13971, 3899, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-100, -100, -100, -100, 4419, -100, 2058, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [85]:
# Let's generate with our real data!

review_data_path = './reviews-word2vec.med.txt' # <- for sanity checking / debugging
#review_data_path = './reviews-word2vec.large.txt.gz' # <- for CPU pre-training and validating
#review_data_path = './reviews-word2vec.larger.txt.gz' <- for GPU pre-training and validating (Part 3.5)


# NOTE: when you eventually deploy this code to Great Lakes, you'll need to use the larger dataset 
# (see the PDF for notes/details)

ofunc = gzip.open if review_data_path.endswith('gz') else open
with ofunc(review_data_path, 'rt') as f:
    reviews = f.readlines()
    reviews = [review.strip() for review in reviews]

dataset = MLMDataset(tokenizer, reviews)
dataset.tokenize()

# get the first item
print(dataset[0])

([101, 2023, 2001, 4149, 2004, 1037, 5592, 1010, 2021, 1996, 2711, 2040, 2288, 2009, 1010, 3866, 2009, 999, 1998, 2027, 103, 103, 2009, 2574, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 2097, 2224, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

## Part 3.2: Create A `collate` function to prepare a batch for training

Once the dataset is ready, we can use the `DataLoader` class  to load the data from the `Dataset` class, much like you did for Homework 2.  Remember that the Dataset class has the __getitem__ method  that returns the input_ids and the labels.
    
Like Homework 2, we'll want to train using a _batch_ of items. This is good for two reasons. First, batching helps us learn from multiple examples at the same time, so the gradient is a bit smoother. Batching provides a good trade-off between SGD (one item at a time) and full GD (all items at once). Second, and perhaps more importantly, batching allows us to maximize the throughput of our computing resources. Depending on the hardware, some matrix operations are the same speed for different sized matrices, so if we can get more examples used to trained per step, this reduces the overall number of steps. You may have seen this in Homework 2 where increasing the batch size dropped the training time, up to some point. 

By default, the `DataLoader` will randomly sample $b$ items from `Dataset` where $b$ is the batch size and turn those into a single `Tensor` to pass as input to the model. To create the batch itself, the `collate` function will tell the `DataLoader` how to turn multiple items into a single `Tensor`.

However, we have a major wrinkle here. When we train BERT for MLM over sequences, not all the sequences have the same length. A batch itself is represented as a `Tensor`. When all the instances in the batch have the same length, we can concatenate/stack them. For example, if we had 10 instances of sequences of length 5, we can create a tensor that is size (10, 5). However, if some of those sequences have different lengths, we no longer can create a Tensor with a single length dimension! What to do?

To solve this, we'll need to write the `collate` function so that it makes all the sequences have the same length. Typically this is done by adding a special `[PAD]` token to the sequences so that they all have the same number of tokens. However, this extra token is meaningless!  If we don't recognize this, then our model will learn to predict `[PAD]` tokens (yikes!). Therefore, we also need to create an _attention mask_ that tells us which tokens to ignore in the input because they are padding. 

In [86]:
def collate_fn(batch: List[Tuple[List[int], List[int]]]):
    '''
    A function that takes a list of instances in the dataset and collates them into a batch.
    '''

    ################################################################
    #                    TODO: YOUR CODE HERE                      #
    # 1. Separate the input_ids and the labels into two lists      #
    # 2. Pad both lists using the tokenizer's padding value so 
    #    that all lists are the same length.
    # 3. Create a boolean attention mask for the input_ids which specifies
    #    which tokens are non-padded elements.
    # 4. Return the padded input_ids, the attention mask, and the padded labels.
    #
    # NOTE: Look at nn.utils.rnn.pad_sequence
    #
    ################################################################

    input_ids, labels = zip(*batch)
    # print(input_ids)
    # print(labels)

    padded_input_ids =  nn.utils.rnn.pad_sequence([torch.tensor(id) for id in input_ids], batch_first=True)
    padded_labels = nn.utils.rnn.pad_sequence([torch.tensor(label) for label in labels], batch_first=True)
    
    attention_mask = (padded_input_ids != 0).int()
    return padded_input_ids, attention_mask, padded_labels

In [87]:
# test the collate function
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# All of the items in the batch should have the same shape!
for input_ids, attention_mask, labels in dataloader:
    print(input_ids[0])
    print(attention_mask[0])
    print(labels[0])
    break

tensor([ 101, 2023, 2001, 4149, 2004, 1037, 5592,  103, 2021,  103, 2711, 2040,
        2288, 2009, 1010, 3866, 2009,  999, 1998,  103,  103, 2224, 2009, 2574,
         102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## Part 3.3: Train BERT for MLM!

Once we have our `Dataset` and `collate` function, it's time to train the pre-model the BERT model for MLM. Working on your laptop to start, try training for 1-2 epochs on the `med` dataset. There's no guarantee the model will learn anything useful, but we can do some experiments later in Part 4 to take a look. 

The core steps for training are the following:
1. Define your model
2. Set up `wandb` for tracking its progress (this will be useful for monitoring when running on Great Lakes)
3. Define the optimizers, learning rate, etc.
4. Define the core training loop

The code won't look too dissimilar from past pytorch training loops in Homeworks 1 and 2.

Training times can vary, but on an M1 Mac and the default hyperparameters, one epoch takes ~50min on the large dataset with "mps" and 6 hours on "cpu". The model and training seem to fit at 12GB of memory. 

For training, we recommend trying to run either the medium or large for a few epochs. If your CPU is slower, try medium just for one epoch to get a sense of what it's learned in Part 4.  For getting a sense of whether the model is working, we strongly recommend testing your CPU-trained model with the masked language modeling task in Task 4.4 to see whether it can correctly fill in common words. The most similar words aren't always a good indicator that it's working.

Don't worry if you can't train too long on your own machine. Once you've gotten the model working (e.g., some preliminary analysis in Part 4 looks "okay" -- doesn't have to be great), it's time to convert this to a script and run it on Great Lakes in Part 3.5 (more details there). You'll use the advanced GPUs on the cluster to go through 5 epochs (or more) during training, which will give you a good enough model that you can use it for the rest of the homework in Parts 4 and 5.


#### VERY OPTIONAL PARTS:

If you are feeling _really_ adventurous, you can try to speed up your model by trying a few of the much fancier things in pytorch. In practice, you should not need these for the homework. However, they can be fun to explore even with a CPU, though they make the biggest impact if you have access to a GPU too

- Try using [torch.compile](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html) to optimize the `nn.Module` code (this tries to pre-compile the computation graph)
- Rather than train with 32-bit floating point, use [amp](https://pytorch.org/blog/what-every-user-should-know-about-mixed-precision-training-in-pytorch/) to do mixed-precision training (i.e., fewer bits and faster). If you want to test this on Great Lakes, the GPUs there support "fp16" which will greatly speed up training. At the moment, `amp` isn't supported on "mps" devices, though it might show up [soon](https://github.com/pytorch/pytorch/issues/88415).
- Implement an alternative training loop using [`accelerate`](https://github.com/huggingface/accelerate) and try using mixed precision (fp8, fp16, bf16) training

In [88]:
print(tokenizer.get_vocab())

{'recess': 28290, 'amateur': 5515, 'tremor': 27734, 'mom': 3566, 'ryan': 4575, 'motorsports': 20711, 'raw': 6315, 'please': 3531, 'occupational': 16928, '##ati': 10450, '।': 1344, 'forbidden': 10386, 'cathedral': 5040, 'hydra': 26018, 'dialogue': 7982, '##kins': 14322, '##zko': 29002, 'unlocked': 14058, 'societe': 18341, 'coding': 16861, 'enthusiasm': 12024, 'dried': 9550, '₇': 1554, 'official': 2880, 'koch': 15259, 'market': 3006, '225': 14993, 'larvae': 9673, 'gazed': 11114, 'pol': 14955, 'goldberg': 18522, 'fire': 2543, 'necks': 26082, '##grapher': 18657, 'bart': 12075, 'lightning': 7407, '133': 14506, '##ards': 18117, 'although': 2348, 'maori': 12600, 'switches': 15924, '##一': 30266, 'arcs': 29137, 'asleep': 6680, 'saigon': 24001, 'hash': 23325, 'ن': 1296, '1852': 8784, 'jordanian': 26276, 'wills': 17234, 'internally': 16058, 'ж': 1186, 'caitlin': 27555, '1856': 8708, 'omitted': 16647, '##logram': 24915, 'º': 1089, 'boulevard': 8459, '221': 19594, '##worm': 22769, 'commodity': 1950

In [90]:
'''
Now, lets put it all together in the training loop.
'''



################################################################
#         TODO: YOUR CODE HERE    
#
# 1. Define the loss function, optimizer, and the BERT model. Use the model hyperparameters from the PDF.
# 2. Specify the hyperparameters for training (e.g., learning rate, batch size, etc.)
# 3. Initialize wandb for logging.
#
################################################################
np.random.seed(305)


batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

epoch_num = 3


learning_rate = 1e-4
hidden_layer_size = 256
embedding_size = 256
vocab_size = len(tokenizer.get_vocab())
num_layers = 2
num_heads = 4
whether_freeze_embedding = False
model = BERT(vocab_size = vocab_size, 
            hidden_size = embedding_size,
            num_heads = num_heads, 
            num_layers = num_layers, 
            num_classes= 2
            )#feed_forward_size = hidden_layer_size)# input
model = model.to(device)

criterion = nn.CrossEntropyLoss()
losses = []
optimizer= optimizer = torch.optim.AdamW(model.parameters(),lr = learning_rate)



#wandb
import wandb
wandb.init(
        # set the wandb project where this run will be logged
        project="hw2--mazhuang",

        # track hyperparameters and run metadata
        config={
        "learning_rate": learning_rate,
        "architecture": "word2vec",
        "dataset": "reviews_word2vec.med",
        "epochs": epoch_num,
        }
    )

# wandb.finish()  
################################################################
#         TODO: YOUR CODE HERE       #
# THe training loop is where we train the model.              #
#                                                             #
# You should implement the following:                         #
# 1. Load the input_ids, attention_mask, and labels to the    #
#    device.                                                  #
# 2. Zero the gradients of the optimizer.                     #
# 3. Get the output from the BERT model.                      #
# 4. Calculate the loss using the output and the labels.      #
# 5. Backpropagate the loss.                                  #
# 6. Update the optimizer.                                    #
# 7. Log the loss to wandb.                                   #
# 8. Save the model every epoch. Use separate files per epoch #
#
# HINT: The tokenizer can be helpful here.
################################################################
steps = 0 
loss_sum = 0
time_begin = time.time()
for epoch in trange(epoch_num, desc="Epoch"):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    for step, (input_ids, attention_mask, labels) in enumerate(tqdm(dataloader, position=1, leave=True, desc="Step")):
        steps += 1

        

        input_ids, labels = input_ids.to(device),  labels.to(device)
        B , T = labels.shape
        #labels = [B,T]
        #attention_mask = attention_mask.unsqueeze(2)
        #attention_mask = attention_mask.unsqueeze(1)
        # #mask = [B,1,T,1]
        
        # attention_mask_3d = attention_mask @ attention_mask.transpose(-2,-1)
        # print(attention_mask_3d.shape)
        #attention_mask = attention_mask.expand(-1, -1, -1, T).float().to(device)
        
        # when tokenizing, I have achieved padding and masking <- not enough
        
        optimizer.zero_grad()

        output , att_weight_col = model(input_ids, attention_mask)

        output = output.view(B * T, -1)
        labels = labels.view(-1)
        #labels = labels.unsqueeze(2)
        

        loss =criterion(output,labels)
        
        loss.backward()
        optimizer.step()

        loss_sum += loss
        #print(step)
        if steps % 100 == 0 :
            #print('loss updated ')
            wandb.log({ "loss": loss_sum}, step = steps)
            losses.append(loss_sum)
            loss_sum = 0 

    path = f'model_zhuangma_epoch{epoch}.pth'
    torch.save(model.state_dict(), path)   
    
    torch.save(att_weight_col, f'att_weight_col_zhuangma{epoch}.json')
    
wandb.finish()  
#model.to('cpu')
model.eval()
time_end = time.time()
time_training =  time_end - time_begin 


# plt.plot(losses.to('cpu'))


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Step:   0%|          | 0/31250 [00:00<?, ?it/s]

Step:   0%|          | 0/31250 [00:00<?, ?it/s]

Step:   0%|          | 0/31250 [00:00<?, ?it/s]

wandb: ERROR Control-C detected -- Run data was not synced


In [ ]:
print(type(att_weight_col[0]))

torch.save(att_weight_col, 'att_weight_col_zhuangma.json')

<class 'torch.Tensor'>


In [ ]:
'''
Now that we have trained the model, we can use it to predict the masked tokens.
'''
model.to(device)
# Predict the masked tokens
def noise_inputs(inputs, mask_token_id, mlm_probability=0.15):
    inputs = deepcopy(inputs)
    labels = [-100] * len(inputs)
    masked_indices = np.random.choice(len(inputs), int(len(inputs) * mlm_probability), replace=False)
    for i in masked_indices:
        if inputs[i] not in [mask_token_id, 101, 102, 0]:
            inputs[i] = mask_token_id
            labels[i] = inputs[i]
    return inputs, labels

def noise_and_predict_tokens(query, tokenizer, model) -> str:
    with torch.no_grad():
        tokenized_input = tokenizer.encode(query)        
        tokens = tokenized_input.tokens
        print('Original:', ' '.join(tokens))
        ids = np.array(tokenized_input.ids)
        inputs, labels = noise_inputs(ids, tokenizer.token_to_id('[MASK]'))
        
        print('Noised: ', ' '.join([tokenizer.id_to_token(at_i) for at_i in inputs]))
        
        response, attns = model(torch.from_numpy(inputs).unsqueeze(0).to(device))
        response = response.argmax(-1).squeeze(0).tolist()
        print('Guess:  ', ' '.join([tokenizer.id_to_token(at_i) for at_i in response[1:-1]]).replace(' ##', ''))

s = 'I really like the book it was great and I loved reading it.'
noise_and_predict_tokens(s, tokenizer, model)

Original: [CLS] i really like the book it was great and i loved reading it . [SEP]
Noised:  [CLS] [MASK] really like the book it was great and i loved reading it . [SEP]
Guess:   wasted are more the book it was had and i like are it .


# Part 3.4: Save the Pre-Trained Model

At this point, save the model's parameters in its `state_dict`. See pytorch's [documentation](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for some guidance here. We'll be using this pre-trained model in later notebooks so once you save it, test that you can load it in another notebook (try `BERT_Inference.ipynb` to start) before moving on. 

In [ ]:
path = f'model_zhuangma_epoch{epoch}.pth'
print(path)

model_zhuangma_epoch0.pth


In [ ]:
path = 'model_zhuangma.pth'
torch.save(model.state_dict(), path)
model_load = BERT(vocab_size = vocab_size, 
            hidden_size = embedding_size,
            num_heads = num_heads, 
            num_layers = num_layers, 
            ).to(device)#feed_forward_size = hidden_layer_size)# input

model_load.load_state_dict(torch.load(path))

noise_and_predict_tokens(s, tokenizer, model_load)


Original: [CLS] i really like the book it was great and i loved reading it . [SEP]
Noised:  [CLS] i [MASK] like the book it was great and i loved reading [MASK] . [SEP]
Guess:   i . characters the book it book no and i characters are poorly .


# Part 3.5 Convert the notebook to a script and submit to Great Lakes for final training

Once you have your model debugged and can verify that it works on a small dataset (manual exploration in Part 4 will help), it's time to train it on more data and for a longer time period. To do this, we'll use the Great Lakes cluster at U-M which will give you access to a GPU that will make training run ~10x faster; this means more epochs and more data in the same amount of time so you get a better model. The Homework PDF has documentation on how use Great Lakes if you haven't seen it. The course account is limited to 4 hours of wallclock time and 16GB of memory, which were tuned specific for this assignment.

Great Lakes supports interactive mode with Jupyter and running a script as a job. We **strongly** encourage the latter. To get a GPU, you'll need to submit a job to the cluster, which uses [SLURM for scheduling](https://arc.umich.edu/greatlakes/slurm-user-guide/). If you attempt to queue for an interactive job, you will have no control over when it starts, so you may end up having your notebook run for 4 hours from 3am to 7am and then it ends, at which point you have to get back in the queue. If you submit a job as a script (i.e., a .py file that runs the code in this notebook), it will run for the specified amount of time and save the BERT model without you having to interact with anything. 

SLURM and cluster scheduling is very common in some industries where there is a single cluster resource and people share it by submitting jobs to run so that no one can monopolize the system and that jobs can run in parallel. Given that Great Lakes will be useful to you in future assignments and projects, we strongly encourage you to learn how to use it effectively in this assignment.

Depending on how you're working on this file, there's a few ways to directly convert the notebook to a file if you use [Jupyter or the command line](https://mljar.com/blog/convert-jupyter-notebook-python/) or [VSCode](https://stackoverflow.com/questions/64297272/best-way-to-convert-ipynb-to-py-in-vscode). Once you convert it, you'll modify the file some to change the epochs and text file as specified in the PDF. **We also strongly recommend having your script save the model at the end of every epoch.**  That way, if your script takes longer than 4 hours and gets killed, you still have the best saved model you could get based on the amount of training you could do.

In [ ]:
from torchsummary import summary
bert = BERT(vocab_size=tokenizer.get_vocab_size(), 
            hidden_size=768, 
            num_heads=4, 
            num_layers=4,
        )


input_data = torch.randint(high= 300,size = (1, 2)).long()

#print(input_data)

summary(bert, 
        #input_data=(1,2,3),
        dtype = torch.long)

Layer (type:depth-idx)                        Param #
├─BertPositionalEmbedding: 1-1                --
|    └─Embedding: 2-1                         23,440,896
|    └─Embedding: 2-2                         393,216
├─ModuleList: 1-2                             --
|    └─TransformerEncoderLayer: 2-3           --
|    |    └─MultiHeadedAttention: 3-1         1,771,776
|    |    └─Sequential: 3-2                   4,722,432
|    └─TransformerEncoderLayer: 2-4           --
|    |    └─MultiHeadedAttention: 3-3         1,771,776
|    |    └─Sequential: 3-4                   4,722,432
|    └─TransformerEncoderLayer: 2-5           --
|    |    └─MultiHeadedAttention: 3-5         1,771,776
|    |    └─Sequential: 3-6                   4,722,432
|    └─TransformerEncoderLayer: 2-6           --
|    |    └─MultiHeadedAttention: 3-7         1,771,776
|    |    └─Sequential: 3-8                   4,722,432
├─MLMHead: 1-3                                --
|    └─Embedding: 2-7                       

Layer (type:depth-idx)                        Param #
├─BertPositionalEmbedding: 1-1                --
|    └─Embedding: 2-1                         23,440,896
|    └─Embedding: 2-2                         393,216
├─ModuleList: 1-2                             --
|    └─TransformerEncoderLayer: 2-3           --
|    |    └─MultiHeadedAttention: 3-1         1,771,776
|    |    └─Sequential: 3-2                   4,722,432
|    └─TransformerEncoderLayer: 2-4           --
|    |    └─MultiHeadedAttention: 3-3         1,771,776
|    |    └─Sequential: 3-4                   4,722,432
|    └─TransformerEncoderLayer: 2-5           --
|    |    └─MultiHeadedAttention: 3-5         1,771,776
|    |    └─Sequential: 3-6                   4,722,432
|    └─TransformerEncoderLayer: 2-6           --
|    |    └─MultiHeadedAttention: 3-7         1,771,776
|    |    └─Sequential: 3-8                   4,722,432
├─MLMHead: 1-3                                --
|    └─Embedding: 2-7                       